In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfTransformer
from keras.regularizers import l2, activity_l2
from keras.callbacks import Callback

class NBatchLogger(Callback):
    def __init__(self):
        self.seen = 0

    def on_epoch_end(self, batch, logs={}):
        self.seen += 1
        if self.seen % 20 == 0:
            print('epoch {}: loss = {}, val = {}'.format(self.seen, logs['loss'], logs['val_loss'])) 

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
train_X = pd.read_json('data/train_scaled_with_leaky_and_added_features_cv.json')
train_y = pd.read_json('data/train_interest.json')[0]
columns = train_X.columns
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=columns)

In [3]:
column_usefulness = [('bc_price', 16597), ('manager_worths', 14684), ('price_by_NTACode', 13343), ('latitude', 12299), ('longitude', 12051), ('desc_words_length', 11477), ('manager_id', 11136), ('price_by_address1', 11125), ('Yday', 10884), ('address1', 10458), ('building_worths', 10298), ('building_id', 10142), ('desc_letters_count', 9832), ('real_minus_expected_price', 9815), ('price_by_area', 9742), ('real_over_expected_price', 9374), ('Day', 9066), ('worth_by_address1', 8782), ('hour', 7722), ('price_per_bathroom', 7590), ('desc_words_count', 7481), ('price_per_bed_and_bath', 7387), ('worth_by_NTACode', 6993), ('photos_count', 6730), ('NTACode', 6324), ('price_by_BoroCode', 6098), ('features_count', 5668), ('Wday', 4445), ('price_per_bedroom', 3903), ('sparse_nofee', 3056), ('bedrooms', 2833), ('sparse_furnished', 1865), ('bathrooms', 1702), ('sparse_laundryinbuilding', 1425), ('sparse_hardwoodfloors', 1132), ('sparse_laundryinunit', 1088), ('Month', 1006), ('sparse_exclusive', 771), ('sparse_dogsallowed', 770), ('BoroCode', 749), ('sparse_catsallowed', 718), ('sparse_prewar', 663), ('sparse_privateoutdoorspace', 634), ('sparse_reducedfee', 631), ('avenue', 595), ('street', 569), ('other_address', 553), ('sparse_commonoutdoorspace', 545), ('Zero_building_id', 534), ('sparse_elevator', 533), ('east', 524), ('sparse_doorman', 510), ('sparse_parkingspace', 506), ('sparse_dishwasher', 500), ('sparse_highspeedinternet', 476), ('sparse_terrace', 424), ('sparse_fitnesscenter', 423), ('sparse_loft', 413), ('sparse_outdoorspace', 406), ('sparse_actualapt', 399), ('sparse_roofdeck', 359), ('sparse_balcony', 346), ('sparse_diningroom', 318), ('sparse_simplex', 292), ('sparse_swimmingpool', 283), ('sparse_shorttermallowed', 263), ('sparse_wheelchairaccess', 248), ('west', 245), ('sparse_newconstruction', 238), ('sparse_garden', 231), ('sparse_multilevel', 201), ('sparse_patio', 199), ('sparse_hardwood', 198), ('worth_by_BoroCode', 179), ('sparse_fireplace', 168), ('sparse_highceiling', 159), ('sparse_petsok', 154), ('sparse_dryerinunit', 153), ('sparse_stainlesssteelappliances', 148), ('sparse_garage', 119), ('south', 118), ('sparse_storage', 118), ('sparse_outdoorareas', 117), ('sparse_lndrybldg', 113), ('sparse_liveinsuper', 110), ('sparse_new', 103), ('sparse_renovated', 99), ('sparse_photos', 88), ('sparse_highceilings', 79), ('sparse_onsitelaundry', 79), ('sparse_view', 78), ('sparse_flex3', 67), ('sparse_centrala', 66), ('sparse_concierge', 64), ('sparse_residentslounge', 63), ('sparse_assignedparkingspace', 62), ('sparse_allutilitiesincluded', 58), ('sparse_onsitegarage', 53), ('north', 46), ('sparse_lowrise', 44), ('sparse_washerinunit', 44)]
leaky = ['manager_worths', 'building_worths', 'price_by_area', 'real_minus_expected_price', 'real_over_expected_price', 'price_by_address1', 'worth_by_address1', 'price_by_BoroCode', 'worth_by_BoroCode', 'price_by_NTACode', 'worth_by_NTACode']
sparse_cols = [x[0] for x in column_usefulness if x[0].startswith('sparse')][:15]
non_sparse_cols = [x[0] for x in column_usefulness if not x[0].startswith('sparse')]

# Add a bunch of features to train_X

In [4]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [5]:
knn_features = [x[0] for x in column_usefulness[:29]]
n_components = 5
new_columns_generated = ['k_means'] + ['pca_{}'.format(i) for i in range(n_components)] + ['knn_{}'.format(i) for i in [2, 4, 8, 16, 64, 256, 1024]]

In [40]:
for column in new_columns_generated:
    train_X_scaled[column] = 0

In [16]:
%%time
tsne = TSNE(n_components = 2)
tsne_representation = tsne.fit_transform(train_X_scaled[knn_features[:15]][:5000])

CPU times: user 1min 3s, sys: 8.51 s, total: 1min 12s
Wall time: 1min 15s


In [41]:
%%time
pca_cols = ['pca_{}'.format(i) for i in range(n_components)]
pca = PCA(n_components=n_components)
pca_representation = pca.fit_transform(train_X_scaled)
train_X_scaled.loc[:, pca_cols] = pca_representation

CPU times: user 1.88 s, sys: 450 ms, total: 2.33 s
Wall time: 1.32 s


In [42]:
%%time 
folds = 5
full_length = len(train_X_scaled)
entries = int(full_length / folds)
full_index = train_X_scaled.index
fold_indices = []
fold_indices_keep = []
fold_indices_throw = []
for column in leaky:
    train_X_scaled[column] = 0
for i in range(folds):
    print ('Fold {}'.format(i))
    indices = sorted(random.sample(list(full_index), entries))
    fold_indices.append(indices)
    full_index = full_index.drop(indices)

    full_index_keep = np.zeros((full_length), dtype=np.bool)
    full_index_throw = np.ones((full_length), dtype=np.bool)
    for j in fold_indices[i]:
        full_index_keep[j] = True
        full_index_throw[j] = False
    fold_indices_throw.append(full_index_throw)
    fold_indices_keep.append(full_index_keep)
    
    #k_means = KMeans(n_clusters = 5)
    #k_means.fit(train_X_scaled[knn_features][full_index_throw], train_y[full_index_throw])
    #k_means_indices = k_means.predict(train_X_scaled[knn_features][full_index_keep])
    #train_X_scaled.loc[indices, 'k_means'] = k_means_indices
    
    #for n_neighbors in [2, 4, 8, 16, 64, 256, 1024]:
    #    knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='manhattan') # mahalanobis
    #    knn.fit(train_X_scaled[knn_features][full_index_throw], train_y[full_index_throw])
    #    knn_pred = knn.predict(train_X_scaled[knn_features][full_index_keep])
    #    train_X_scaled.loc[indices, 'knn_{}'.format(n_neighbors)] = knn_pred

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
CPU times: user 27min 13s, sys: 9.89 s, total: 27min 23s
Wall time: 27min 48s


In [43]:
train_X_scaled.to_json('data/train_scaled_with_leaky_and_added_features_cv.json', orient='records')

# Train Models

In [ ]:
%%time
train_X_cv = pd.read_json('data/train_with_leaky_cv.json')
train_X_scaled_cv = pd.DataFrame(StandardScaler().fit_transform(train_X_cv), columns=train_X_cv.columns)

ntrain = int(len(train_X_cv) * 0.75)

xtr = train_X_scaled_cv[:ntrain]
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X_scaled_cv[ntrain:]
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

def create_model(xtr):
    model = Sequential()
    
    model.add(Dense(500, input_dim=xtr.shape[1], init = 'he_normal', activation='sigmoid'))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.75))
    model.add(PReLU())

    model.add(Dense(100, init = 'he_normal', activation='sigmoid'))
    model.add(BatchNormalization())    
    model.add(Dropout(0.75))
    model.add(PReLU())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

filepath = 'data/nn_weights'

logger = NBatchLogger()
early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
model = create_model(xtr)
model.fit(xtr.values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte.values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])
model = create_model(xtr)
model.load_weights(filepath)

epoch 20: loss = 0.6258024382212498, val = 0.6258474560374475
epoch 40: loss = 0.6015593465094552, val = 0.6047516582676533
epoch 60: loss = 0.5855522775576594, val = 0.5948427323391458
epoch 80: loss = 0.5746987507648224, val = 0.588205450501738
epoch 100: loss = 0.5684100572236606, val = 0.5846041348402148
epoch 120: loss = 0.5586724175190484, val = 0.5812287285795937


In [30]:
train_X = pd.read_json('data/train_leaky.json')
train_X_scaled = pd.DataFrame(StandardScaler().fit_transform(train_X), columns=train_X.columns)

In [31]:
ntrain = int(len(train_X) * 0.75)

xtr = train_X_scaled[:ntrain]
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X_scaled[ntrain:]
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

In [32]:
#new_vects = ['tf_idf_' + x[7:] for x in sparse_cols]
#sparse = processed_train[sparse_cols]
#tf_idfs = TfidfTransformer().fit_transform(sparse)
#tf_idfs_df = pd.DataFrame(tf_idfs.toarray(), columns=new_vects)
#processed_train = processed_train.join(tf_idfs_df, how='left')

# Neural Net

In [33]:
def create_model(xtr):
    model = Sequential()
    
    model.add(Dense(500, input_dim=xtr.shape[1], init = 'he_normal', activation='sigmoid'))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.75))
    model.add(PReLU())

    model.add(Dense(100, init = 'he_normal', activation='sigmoid'))
    model.add(BatchNormalization())    
    model.add(Dropout(0.75))
    model.add(PReLU())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

In [34]:
%%time
filepath = 'data/nn_weights'

logger = NBatchLogger()
early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
model = create_model(xtr)
model.fit(xtr.values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte.values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])
model = create_model(xtr)
model.load_weights(filepath)

epoch 20: loss = 0.6089174983444364, val = 0.6067765715308119
epoch 40: loss = 0.5853107286090885, val = 0.5850755318499284
epoch 60: loss = 0.5705643378497265, val = 0.5771795387089513
epoch 80: loss = 0.5579046620334922, val = 0.5707317711294789
epoch 100: loss = 0.5534765775343996, val = 0.5669029398624583
epoch 120: loss = 0.546212861740448, val = 0.5643390875172627
epoch 140: loss = 0.5400573863694326, val = 0.5620383667605761
epoch 160: loss = 0.5329683061201813, val = 0.5618096001435378
epoch 180: loss = 0.5292632116061488, val = 0.5612778273195361
epoch 200: loss = 0.5248733873195198, val = 0.5598880919534396
epoch 220: loss = 0.5186089570991107, val = 0.5604438877101848
epoch 240: loss = 0.5180035672343942, val = 0.5601027862993587
epoch 260: loss = 0.5172649729823205, val = 0.5597893895989281
epoch 280: loss = 0.5114319491774181, val = 0.5599595837972601
CPU times: user 22min 33s, sys: 2min 25s, total: 24min 59s
Wall time: 18min 28s


In [ ]:
model = create_model(xtr)
model.load_weights('data/nn_weights_not_cv')
nn_predictions = model.predict(xte.values)
print (log_loss(yte_one_hot, nn_predictions))
test_X = pd.read_json('data/test_leaky.json')
test_X = pd.DataFrame(scaler.transform(test_X), columns=columns)
listing_id = pd.read_json('data/test_ids.json').valuespreds = model.predict(test_X.values)
preds = pd.DataFrame(preds)
preds.columns = ['high', 'medium', 'low']
preds['listing_id'] = listing_id
preds.to_csv('data/my_preds_not_cv.csv', index=None)

In [ ]:
model = create_model(xtr)
model.load_weights('data/nn_weights')
nn_predictions = model.predict(xte.values)
print (log_loss(yte_one_hot, nn_predictions))
test_X = pd.read_json('data/test_leaky.json')
test_X = pd.DataFrame(scaler.transform(test_X), columns=columns)
listing_id = pd.read_json('data/test_ids.json').valuespreds = model.predict(test_X.values)
preds = pd.DataFrame(preds)
preds.columns = ['high', 'medium', 'low']
preds['listing_id'] = listing_id
preds.to_csv('data/my_preds_cv.csv', index=None)

In [19]:
nn_predictions = model.predict(xte.values)
log_loss(yte_one_hot, nn_predictions)
#0.56359369155527683 before adding pca, knn, mkeans to predictions

0.60016923349431917

# Ridge Lasso Elastic

In [307]:
clf = RidgeClassifier()
clf.fit(xtr, ytr)
print (clf.score(xte, yte))
predictions = clf.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[predictions]))

0.726779056573
9.43671707143


In [308]:
las = Lasso()
enet = ElasticNet()
las.fit(xte, yte)
enet.fit(xte, yte)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [309]:
print (las.score(xte, yte))
predictions = las.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [310]:
print (enet.score(xte, yte))
predictions = enet.predict(xte)
print (log_loss(yte_one_hot, np.eye(3)[np.round(predictions).astype(int)]))

0.0
10.478087214


In [42]:
for coef, column in zip(enet.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

In [43]:
for coef, column in zip(las.coef_, train_X.columns):
    if coef != 0:
        print (column, ": ", coef)

# TREES

In [322]:
test = ExtraTreesClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, predictions)))

0.5843451108603969 (test loss 0.3912286072900344)


In [323]:
test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
et_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_ent_predictions), log_loss(ytr_one_hot, predictions)))

0.5841850290743527 (test loss 0.3945679429419886)


In [335]:
test = RandomForestClassifier(
    n_estimators=100,
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
rf_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_predictions), log_loss(ytr_one_hot, predictions)))

0.5693372215730407 (test loss 0.3780482702094549)


In [336]:
test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)

test.fit(xtr, ytr)
            
predictions = test.predict_proba(xtr)
rf_ent_predictions = test.predict_proba(xte)
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions), log_loss(ytr_one_hot, predictions)))

0.5668175898671493 (test loss 0.3773447326368733)


In [337]:
pred_mixed = (0.01 * et_predictions + 0.01 * et_ent_predictions + 0.49 * rf_predictions + 0.49 * rf_ent_predictions)
log_loss(yte_one_hot, pred_mixed)

0.56688904952650898

In [11]:
from sklearn.neighbors import KNeighborsClassifier

In [12]:
%%time
test = KNeighborsClassifier(n_neighbors=2)
test.fit(xtr[non_sparse_cols], ytr)

predictions = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions = test.predict_proba(xte[non_sparse_cols])

7.076257047822155 (test loss 0.23129250627049086)
CPU times: user 3min 15s, sys: 1.48 s, total: 3min 16s
Wall time: 3min 20s


In [13]:
%%time
test = KNeighborsClassifier(n_neighbors=5)
test.fit(xtr[non_sparse_cols], ytr)

predictions_2 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_2 = test.predict_proba(xte[non_sparse_cols])

2.997161309463761 (test loss 0.4189513500335195)
CPU times: user 3min 36s, sys: 1.02 s, total: 3min 37s
Wall time: 3min 38s


In [14]:
%%time
test = KNeighborsClassifier(n_neighbors=10)
test.fit(xtr[non_sparse_cols], ytr)

predictions_3 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_3 = test.predict_proba(xte[non_sparse_cols])

1.5474359459916005 (test loss 0.4994428601539661)
CPU times: user 3min 48s, sys: 839 ms, total: 3min 49s
Wall time: 3min 50s


In [15]:
%%time
test = KNeighborsClassifier(n_neighbors=20)
test.fit(xtr[non_sparse_cols], ytr)

predictions_4 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_4 = test.predict_proba(xte[non_sparse_cols])

CPU times: user 4min 9s, sys: 1.36 s, total: 4min 11s
Wall time: 4min 15s


In [16]:
%%time
test = KNeighborsClassifier(n_neighbors=50)
test.fit(xtr[non_sparse_cols], ytr)

predictions_5 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_5 = test.predict_proba(xte[non_sparse_cols])

CPU times: user 4min 18s, sys: 1.2 s, total: 4min 19s
Wall time: 4min 21s


In [18]:
%%time
test = KNeighborsClassifier(n_neighbors=50)
test.fit(xtr[non_sparse_cols], ytr)

predictions_45 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_45 = test.predict_proba(xte[non_sparse_cols])

CPU times: user 4min 17s, sys: 1.06 s, total: 4min 18s
Wall time: 4min 19s


In [17]:
%%time
test = KNeighborsClassifier(n_neighbors=100)
test.fit(xtr[non_sparse_cols], ytr)

predictions_5 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_5 = test.predict_proba(xte[non_sparse_cols])

CPU times: user 4min 26s, sys: 1.07 s, total: 4min 27s
Wall time: 4min 28s


http://mlwave.com/kaggle-ensembling-guide/
https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14295
https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335
http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html
http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [21]:
%%time
test = KNeighborsClassifier(n_neighbors=200)
test.fit(xtr[non_sparse_cols], ytr)

predictions_6 = test.predict_proba(xtr[non_sparse_cols])
rf_ent_predictions_6 = test.predict_proba(xte[non_sparse_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions_6), log_loss(ytr_one_hot, predictions_6)))

0.6344317626130038 (test loss 0.6195293255389913)
CPU times: user 4min 34s, sys: 1.19 s, total: 4min 35s
Wall time: 4min 36s


In [20]:
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions_45), log_loss(ytr_one_hot, predictions_45)))

0.7014805929988618 (test loss 0.5857152625149388)


In [19]:
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_ent_predictions_5), log_loss(ytr_one_hot, predictions_5)))

0.6436552097433358 (test loss 0.6045059334208441)


In [ ]:
for feat, importance in zip(processed_train[cols].columns, clf.feature_importances_):
    if not np.isnan(importance) and importance > 0:
        print (feat, ": ", importance)

In [345]:
print ("nn: ", log_loss(yte_one_hot, nn_predictions))
print ("ab: ", log_loss(yte_one_hot, pred_mixed))
frac_nn = 0.2
averaged_preds = (nn_predictions * frac_nn) + (pred_mixed * (1 - frac_nn))
print ("combo: ", log_loss(yte_one_hot, averaged_preds))

nn:  0.563593691555
ab:  0.566889049527
combo:  0.55827286106


In [ ]:
train_X = pd.read_json('data/train_leaky.json')
train_y = pd.read_json('data/train_interest.json')[0]
scaler = StandardScaler().fit(train_X)
train_X = pd.DataFrame(scaler.transform(train_X), columns=columns)
test_X = pd.read_json('data/test_leaky.json')
test_X = pd.DataFrame(scaler.transform(test_X), columns=columns)
listing_id = pd.read_json('data/test_ids.json').values
preds = pd.read_csv('data/my_best_preds.csv')

In [295]:
preds = pd.read_csv('data/my_best_preds.csv')

In [313]:
et_clf = ExtraTreesClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='auto',
    min_samples_split=20)
et_clf.fit(train_X, train_y)          
et_predictions = et_clf.predict_proba(test_X)

In [314]:
et_clf_ent = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=20)
et_clf_ent.fit(train_X, train_y)          
et_ent_predictions = et_clf_ent.predict_proba(test_X)

In [316]:
rf_clf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_split=20)
rf_clf.fit(train_X, train_y)          
rf_predictions = rf_clf.predict_proba(test_X)

In [317]:
rf_clf_ent = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    min_samples_split=20)
rf_clf_ent.fit(train_X, train_y)          
rf_ent_predictions = rf_clf_ent.predict_proba(test_X)

In [259]:
nn_predictions = model.predict(test_X.values)

In [297]:
xgb_predictions = preds[['high', 'medium', 'low']].values

In [303]:
new_preds = (0.8 * xgb_predictions) + (0.15 * nn_predictions) + (0.05 * et_predictions)

In [304]:
new_preds_df = pd.DataFrame(new_preds, columns=['high', 'medium', 'low'])
new_preds_df['listing_id'] = preds['listing_id']

In [300]:
preds.head()

,high,medium,low,listing_id
0,0.062057,0.562695,0.375248,7142618
1,0.072976,0.033289,0.893735,7210040
2,0.000501,0.004300,0.995199,6832604
3,0.130962,0.298393,0.570644,6830595
4,0.002368,0.008273,0.989359,6843709


In [305]:
new_preds_df.head()

,high,medium,low,listing_id
0,0.057388,0.528401,0.414212,7142618
1,0.059807,0.031273,0.908920,7210040
2,0.000814,0.005621,0.993565,6832604
3,0.129824,0.305827,0.564349,6830595
4,0.002757,0.011676,0.985567,6843709


In [306]:
new_preds_df.to_csv('data/combined_preds.csv', index=None)